In [1]:
from os.path import dirname, abspath, join
from warnings import warn

import numpy as np
import scipy as sp
from scipy import io
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.decomposition import NMF
from time import perf_counter as pc

import matplotlib.pyplot as plt
plt.switch_backend('agg')
try:
    from matplotlib.pyplot import savefig, imshow, set_cmap
except ImportError as exc:
    warn("Matplotlib must be installed.")

In [311]:
def read(inputX):
        data = io.mmread(inputX)
        return data

In [28]:
inputF = "../00.data/CEMBA171212_4B/tmp.repl1_CEMBA171212_4B.sparse.filtered.mmtx" 
V = read(inputF)

In [129]:
from scipy.stats import describe
from scipy.stats.mstats import mquantiles
from numpy import array, ravel
from scipy.sparse import save_npz
from scipy.sparse import load_npz
V = load_npz("../00.data/CEMBA171212_4B/tmp.repl1_CEMBA171212_4B.sparse.npz")
print(V.tocsr().shape)
V = (V*1).tocsr().T
print(V.shape)
rowSum = np.sum(V,axis=0)
colSum = np.sum(V,axis=1)
describe(rowSum.T)
describe(colSum)
V.shape

(7324, 198851)
(198851, 7324)


(198851, 7324)

In [48]:
# read in x/y axis

def read_xgi(xgi):
        fname = xgi
        with open(fname) as f:
                xgi_content = f.readlines()
                # you may also want to remove whitespace characters like `\n` at the end of each line
        xgi_content = [x.strip() for x in xgi_content]
        return xgi_content
    
def read_ygi(ygi):
        fname = ygi
        with open(fname) as f:
                ygi_content = f.readlines()
                # you may also want to remove whitespace characters like `\n` at the end of each line
        ygi_content = [x.strip() for x in ygi_content]
        return ygi_content

xgi = read_xgi("../00.data/CEMBA171212_4B/tmp.repl1_CEMBA171212_4B.xgi")
ygi = read_ygi("../00.data/CEMBA171212_4B/tmp.repl1_CEMBA171212_4B.ygi")

In [51]:
from scipy.stats import describe
from scipy.stats.mstats import mquantiles
from numpy import array, ravel
prob = 0.05
n = 1000
print(V.shape)
colSum = np.sum(V,axis=0)
print(describe(colSum.T))
print(colSum.shape)
col_tokeep = ravel(colSum >= n)
print(col_tokeep.T.shape)
newV = V[:, col_tokeep]
print(newV.shape)

rowSum = np.sum(newV,axis=1)
print(describe(rowSum))
rowQ = mquantiles(np.array(rowSum), prob=prob)
row_tokeep = ravel(rowSum >= rowQ)
print(row_tokeep.shape)
newV = newV[row_tokeep, :]
print(newV.shape)

(198851, 7324)
DescribeResult(nobs=7324, minmax=(array([136]), array([34685])), mean=array([2949.94797925]), variance=array([11600051.43099551]), skewness=array([2.62837745]), kurtosis=array([9.86830481]))
(1, 7324)
(7324,)
(198851, 4908)
DescribeResult(nobs=198851, minmax=(array([1], dtype=int64), array([1867], dtype=int64)), mean=array([101.39307321]), variance=array([8655.33944792]), skewness=array([2.75193436]), kurtosis=array([12.16240947]))
(198851,)
(190187, 4908)


In [40]:
col_tokeep

array([ True,  True,  True, ...,  True,  True,  True])

In [52]:
from itertools import compress
selt_xgi = list(compress(xgi, col_tokeep))
selt_ygi = list(compress(ygi, row_tokeep))
len(selt_xgi)
len(selt_ygi)

190187

In [25]:
print(row_tokeep[0:20])

[ True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True]


In [130]:
def filter_V(V, xgi, ygi, prob, ct):
        colSum = np.sum(V,axis=0)
        """print(describe(colSum.T))"""
        col_tokeep = ravel(colSum >= ct)
        newV = V[:, col_tokeep]

        rowSum = np.sum(newV,axis=1)
        rowQ = mquantiles(np.array(rowSum), prob=prob)
        row_tokeep = ravel(rowSum >= rowQ)
        newV = newV[row_tokeep, :]
        selt_xgi = list(compress(xgi, col_tokeep))
        selt_ygi = list(compress(ygi, row_tokeep))
        return {'V':newV, 'xgi':selt_xgi, 'ygi':selt_ygi}
out = filter_V(V, xgi, ygi, 0.05, 1000)

In [137]:
selt_ygi = out['ygi']
np.savetxt('.'.join(["../foo.tfidf.nndsvd.r6.n30", "ygi"]), selt_ygi, fmt= "%s", delimiter="\t")

In [142]:
newV = out['V']
newV.todense()

(190187, 4908)

In [128]:
V = newV
print(type(V))
print(V.shape)
for row, col, value in zip(V.tocoo().row[0:20], V.tocoo().col[0:20], V.tocoo().data[0:20]):
    out = "{0}\t{1}\t{2}".format(row, col, value)
    print(out)
mat = V.tocsr().T.astype('float32')
print(mat.shape)
tfidf = TfidfTransformer()
mat = tfidf.fit_transform(mat)
mat = mat.T.tocoo()
print(mat.shape)
for row, col, value in zip(mat.row[0:20], mat.col[0:20], mat.data[0:20]):
    out = "{0}\t{1}\t{2}".format(row, col, value)
    print(out)

<class 'scipy.sparse.csc.csc_matrix'>
(190187, 4908)
117	0	2
120	0	2
272	0	2
311	0	2
339	0	2
483	0	2
513	0	2
598	0	2
638	0	1
778	0	2
945	0	2
985	0	2
1236	0	2
1570	0	2
1621	0	1
1703	0	2
1762	0	2
1806	0	2
1921	0	2
2195	0	2
(4908, 190187)


/mnt/silencer2/home/yangli/apps/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(190187, 4908)
189976	0	0.021169063491889187
189975	0	0.020678824199565002
189973	0	0.02048989381388413
189928	0	0.01841534784303461
189870	0	0.024528185881795
189779	0	0.020169316197318623
189702	0	0.02108340092582712
189541	0	0.017719192329309404
189469	0	0.027165920904532497
189448	0	0.02583579798643279
189415	0	0.0262353881781657
189346	0	0.02481992460338233
189131	0	0.02683300143016552
188873	0	0.02303255208137045
188818	0	0.024720476126460365
188704	0	0.04850977087544142
188588	0	0.024167492198210842
188543	0	0.02399783052245216
188536	0	0.023677252905886657
188482	0	0.01769975041691977


In [84]:
foo = np.array([[1,2,3],[4,5,6],[1,1,1],[0,1,1],[0,0,1]])
print(foo.shape)
print(foo)
tfidf = TfidfTransformer()
foo2 = tfidf.fit_transform(foo.T)
foo2 = foo2.T.tocoo()
print(foo2.todense())
print(foo2.shape)

for row, col, value in zip(foo2.row, foo2.col, foo2.data):
    out = "{0}\t{1}\t{2}".format(row, col, value)
    print(out)

(5, 3)
[[1 2 3]
 [4 5 6]
 [1 1 1]
 [0 1 1]
 [0 0 1]]
[[0.23570226 0.35545727 0.4220546 ]
 [0.94280904 0.88864317 0.84410921]
 [0.23570226 0.17772863 0.14068487]
 [0.         0.22885798 0.18115738]
 [0.         0.         0.23820019]]
(5, 3)
2	0	0.23570226039551587
1	0	0.9428090415820635
0	0	0.23570226039551587
3	1	0.22885797652116943
2	1	0.17772863458867436
1	1	0.8886431729433717
0	1	0.3554572691773487
4	2	0.23820018731109147
3	2	0.18115738216786942
2	2	0.1406848678283926
1	2	0.8441092069703555
0	2	0.42205460348517776


/mnt/silencer2/home/yangli/apps/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [122]:
V = mat
rank = 6
print(V.shape[0])
print(V.shape[1])
rowN = V.shape[0]
colN = V.shape[1]
initW = np.random.random((rowN,rank))
initH = np.random.random((rank,colN))
model = NMF(n_components=rank, init='random', random_state=2, verbose=True)

190187
4908


In [123]:
W = model.fit_transform(V)
H = model.components_

violation: 1.0
violation: 0.7373206262039805
violation: 0.7020395558833484
violation: 0.5227788238055112
violation: 0.3531165919918594
violation: 0.23570546437023424
violation: 0.18712276846245662
violation: 0.1552134590499713
violation: 0.12298733299284502
violation: 0.09459385189808159
violation: 0.07295398258492004
violation: 0.05714069186326924
violation: 0.04567685173225137
violation: 0.03717593374877272
violation: 0.03042241669532991
violation: 0.025043618396431655
violation: 0.020536494791931998
violation: 0.016912941645902103
violation: 0.013960665588550268
violation: 0.011522697993707125
violation: 0.009552609535076707
violation: 0.007940181662374115
violation: 0.0066366955611333
violation: 0.0055573100359790914
violation: 0.004658427275379151
violation: 0.003905086833756713
violation: 0.003276565998968257
violation: 0.002747099363743538
violation: 0.0023124403632814932
violation: 0.0019504599504565014
violation: 0.0016454878482369839
violation: 0.0013875440635163544
violation

In [97]:
W.shape

(190187, 6)

In [124]:
def predict_H(H):
    colmax = np.amax(H, axis=0)
    colsum = np.sum(H, axis=0)
    p = colmax / colsum
    idx = H.argmax(axis=0)
    out = [idx, p]
    return out
idx = predict_H(H)[0]
idx[0:20]

array([0, 3, 5, 5, 3, 5, 2, 4, 1, 0, 3, 5, 1, 2, 0, 4, 3, 2, 3, 3])

In [99]:
xidx = list(np.concatenate(np.where(idx == 7)))
len(xidx)

0

In [120]:
import itertools

def cal_connectivity(H, idx):
    connectivity_mat = np.zeros((H.shape[1], H.shape[1]))
    classN = H.shape[0]
    for i in range(classN):
        xidx = list(np.concatenate(np.where(idx == i)))
        iterables = [ xidx, xidx ]
        for t in itertools.product(*iterables):
            connectivity_mat[t[0],t[1]] = 1
    return connectivity_mat
connectivity_mat = cal_connectivity(H, idx)

In [121]:
connectivity_mat[0:15,0:15]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0.

In [443]:
def plot(prefix, C, rank):
        """
        Plot reordered consensus matrix.

        :param C: Reordered consensus matrix.
        :type C: numpy.ndarray`
        :param rank: Factorization rank.
        :type rank: `int`
        """
        fig = plt.figure(figsize=(5,5), dpi=100);
        imshow(C)
        set_cmap("RdBu_r")
        fileN = [prefix, "C", "png"]
        fileN = '.'.join(fileN)
        fig.savefig(fileN)

def reorder(C):
        """
        Reorder consensus matrix.

        :param C: Consensus matrix.
        :type C: `numpy.ndarray`
        """
        Y = 1 - C
        Z = linkage(squareform(Y), method='average')
        ivl = leaves_list(Z)
        ivl = ivl[::-1]
        return C[:, ivl][ivl, :]

In [452]:
print(connectivity_mat.shape)
plot("./C", connectivity_mat, 3)

(4908, 4908)


In [194]:
xidx = list(np.where(idx == 7))
iterables = [ list(xidx), list(xidx) ]
for t in itertools.product(*iterables):
    print(t)

(array([   0,    9,   10, ..., 4903, 4904, 4906]), array([   0,    9,   10, ..., 4903, 4904, 4906]))


In [185]:
connectivity_mat[0:15,0:15]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.

In [66]:
import cmath
vec = list(np.concatenate(W))
absSum = np.sum(np.abs(vec))
n = np.prod(H.shape)
np.sqrt(n)
np.sqrt(np.sum(np.square(vec)))

array([0.00000000e+00, 0.00000000e+00, 2.79142458e-03, ...,
       4.33374628e-04, 1.09049062e-04, 5.09863859e-05])

In [69]:
def cal_sparseness(X):
    vec = list(np.concatenate(X))
    absSum = np.sum(np.abs(vec))
    n = np.prod(X.shape)
    squareSum = np.sum(np.square(vec))
    numerator = np.sqrt(n) - (absSum / np.sqrt(squareSum))
    denominator = np.sqrt(n) - 1
    sparseness = numerator / denominator
    return sparseness

cal_sparseness(W)

0.5777956422284521

In [75]:
model.reconstruction_err_

6451.831206079785

In [107]:
def cal_rss(W, H, V):
    rss = np.sum(np.square(W.dot(H) - V))
    return rss
rss = cal_rss(W,H,V)

In [113]:
def cal_mse(W, H, V):
    """ # The mean square error """
    mse = np.mean(np.square(W.dot(H) - V))
    return mse
cal_mse(W, H, V)

0.044582005614562215

In [102]:
def cal_evar(rss, V):
    evar = 1 - ( rss / np.sum(V.data**2))
    return evar
cal_evar(rss, V)

0.0494019487145142

In [ ]:
def cal_cophenetic(C):
    X = C  # Original data (1000 observations)
    Z = fc.linkage_vector(X)         # Clustering
    orign_dists = fc.pdist(X)  # Matrix of original distances between observations
    cophe_dists = cophenet(Z)  # Matrix of cophenetic distances between observations
    corr_coef = np.corrcoef(orign_dists, cophe_dists)[0,1]
    return corr_coef

In [269]:
def cal_dispersion(C):
    n = C.shape[1]
    corr_disp = np.sum(4 * np.square(np.concatenate(C - 1/2)))/(np.square(n))
    return corr_disp
cal_dispersion(connectivity_mat)

1.0

In [268]:
np.sum(4 * np.square(np.concatenate(connectivity_mat - 1/2)))/(np.square(connectivity_mat.shape[1]))

1.0

In [241]:
import math
k = W.shape[1]
m = W.shape[0]
s_list = []
for i in range(m):
    rowsum = np.sum(W[i,])
    p_iq_x_list = []
    for q in range(k):
        p_iq = W[i,q] / rowsum
        if p_iq != 0:
            tmp = p_iq * math.log(p_iq,2)
        else:
            tmp = 0
        p_iq_x_list.append(tmp)
    s = 1 + 1/math.log(k,2) * np.sum(p_iq_x_list)
    s_list.append(s)
s_list

[0.37061973893284095,
 0.24672219246652616,
 0.3468616411022849,
 0.27046688663682894,
 0.2063036470211118,
 0.30387656052137657,
 0.42637071447713304,
 0.44685306696484517,
 0.2596019545321191,
 0.5353673466288251,
 0.23639609817946194,
 0.40639546869330245,
 0.2620021018866523,
 0.3385235970439372,
 0.5948750552557125,
 0.704258209181492,
 0.29483912250721334,
 0.4708615699178681,
 0.10802806190076619,
 0.5066283830026465,
 0.29456523174715254,
 0.5027179635260839,
 0.26638724987907947,
 0.29892612385709716,
 0.4271430361522155,
 0.3814515805631643,
 0.27748009823299435,
 0.28829338091881895,
 0.4818190580848407,
 0.2985874180068171,
 0.5146199160097125,
 0.6133526541369573,
 0.5620648983491132,
 0.4789366132625088,
 0.20331744794648632,
 0.5363934979055538,
 0.39723740421404186,
 0.3098590933132568,
 0.3766750959742232,
 0.2671239008630797,
 0.29886225870313543,
 0.31240225404729516,
 0.7241157532680539,
 0.5270002173380894,
 0.24597168256137447,
 0.4156687483329984,
 0.308919382326

In [237]:
rowsum = np.sum(W[0,])
p_iq = W[0,2] / rowsum
p_iq * math.log(p_iq,2)

-0.529028275480786

In [23]:
%timeit W = model.fit_transform(V)
H = model.components_

violation: 1.0
violation: 0.4260195299480686
violation: 0.3525898106551261
violation: 0.26798858215325533
violation: 0.19227394815619364
violation: 0.1481921690401515
violation: 0.13589253398488935
violation: 0.13753372286528098
violation: 0.13615726106364
violation: 0.11917166055655586
violation: 0.0956021418963788
violation: 0.07677709521030329
violation: 0.06296776628358926
violation: 0.05226201367092705
violation: 0.043731185008408266
violation: 0.03654933120380306
violation: 0.030692054930010416
violation: 0.025987937255566952
violation: 0.021986406996506787
violation: 0.018471622595821
violation: 0.016087725810343626
violation: 0.01413135672441644
violation: 0.012566988510043864
violation: 0.011218324650078923
violation: 0.010060898312027243
violation: 0.009070551309075038
violation: 0.008213772220720146
violation: 0.007480408966762481
violation: 0.006840388511462167
violation: 0.00627298251780503
violation: 0.005764666593621284
violation: 0.005315693795623424
violation: 0.004912

violation: 0.00032221039677373574
violation: 0.0003149569178996609
violation: 0.0003079574696625409
violation: 0.00030123794963528587
violation: 0.000294776643161886
violation: 0.00028844507238796495
violation: 0.00028237345609775065
violation: 0.00027649981328462317
violation: 0.00027077735595091944
violation: 0.00026522552666376637
violation: 0.0002598275129176516
violation: 0.00025458106385417497
violation: 0.0002494891174809499
violation: 0.00024450289704335514
violation: 0.00023965517184231972
violation: 0.0002349287744736749
violation: 0.00023030283942337374
violation: 0.00022577986123591346
violation: 0.00022135882925179432
violation: 0.00021705400165042171
violation: 0.00021284042992396435
violation: 0.00020871987212590676
violation: 0.00020468476808638744
violation: 0.00020073265213143773
violation: 0.00019685929068311275
violation: 0.00019306761542692207
violation: 0.00018936348916173877
violation: 0.00018573889428444895
violation: 0.00018218791661878947
violation: 0.00017871

violation: 0.004551450015599615
violation: 0.004223938828203297
violation: 0.003924498663176398
violation: 0.0036523824497112904
violation: 0.003402951401630589
violation: 0.0031775455391290825
violation: 0.002966561230696171
violation: 0.002774337932289851
violation: 0.0025968278032230067
violation: 0.0024324337183528337
violation: 0.0022854205007771804
violation: 0.002152379346899065
violation: 0.0020278886996673183
violation: 0.001914850846488044
violation: 0.0018116243657153087
violation: 0.001716397110447084
violation: 0.0016259744151233648
violation: 0.001546543165833481
violation: 0.0014733994273610205
violation: 0.0014053800758656932
violation: 0.0013426740009503488
violation: 0.001284442705860001
violation: 0.0012300937130026587
violation: 0.001179159294254522
violation: 0.001131286590364284
violation: 0.001086201646748481
violation: 0.0010436709908796722
violation: 0.0010032584425355522
violation: 0.0009651483514131589
violation: 0.0009288725777345247
violation: 0.00089458486

In [25]:
start_time = pc()
%prun model.fit_transform(data)
end_time = pc()

violation: 1.0
violation: 0.4260195299480686
violation: 0.3525898106551261
violation: 0.26798858215325533
violation: 0.19227394815619364
violation: 0.1481921690401515
violation: 0.13589253398488935
violation: 0.13753372286528098
violation: 0.13615726106364
violation: 0.11917166055655586
violation: 0.0956021418963788
violation: 0.07677709521030329
violation: 0.06296776628358926
violation: 0.05226201367092705
violation: 0.043731185008408266
violation: 0.03654933120380306
violation: 0.030692054930010416
violation: 0.025987937255566952
violation: 0.021986406996506787
violation: 0.018471622595821
violation: 0.016087725810343626
violation: 0.01413135672441644
violation: 0.012566988510043864
violation: 0.011218324650078923
violation: 0.010060898312027243
violation: 0.009070551309075038
violation: 0.008213772220720146
violation: 0.007480408966762481
violation: 0.006840388511462167
violation: 0.00627298251780503
violation: 0.005764666593621284
violation: 0.005315693795623424
violation: 0.004912

In [257]:
import fastcluster as fc
import numpy as np
from scipy.cluster.hierarchy import cophenet

X = np.random.random((10,10))  # Original data (1000 observations)
Z = fc.linkage_vector(X)         # Clustering

orign_dists = fc.pdist(X)  # Matrix of original distances between observations
cophe_dists = cophenet(Z)  # Matrix of cophenetic distances between observations

# What I really want at the end of the day is
corr_coef = np.corrcoef(orign_dists, cophe_dists)[0,1]

In [260]:
import fastcluster as fc
import numpy as np
from scipy.cluster.hierarchy import cophenet

X = connectivity_mat  # Original data (1000 observations)
Z = fc.linkage_vector(X)         # Clustering

orign_dists = fc.pdist(X)  # Matrix of original distances between observations
cophe_dists = cophenet(Z)  # Matrix of cophenetic distances between observations

# What I really want at the end of the day is
corr_coef = np.corrcoef(orign_dists, cophe_dists)[0,1]
corr_coef

0.9875639249148379

In [309]:
X = np.random.random((100,50))
rank = 8
rowN = X.shape[0]
colN = X.shape[1]
initW = np.random.random((rowN,rank))
initH = np.random.random((rank,colN))
model = NMF(n_components=8, init='custom', random_state=0, verbose=True)
W = model.fit_transform(X, H=initH, W=initW)
H = model.components_

violation: 1.0
violation: 0.027185549039051356
violation: 0.03299287723857403
violation: 0.02694724195826974
violation: 0.02435759595044415
violation: 0.021998354238316687
violation: 0.020465466775329375
violation: 0.018625216906039795
violation: 0.016914862928601546
violation: 0.015580073129088755
violation: 0.014271785102295112
violation: 0.012864128042797853
violation: 0.011687687352055075
violation: 0.01026874760871942
violation: 0.009024534429585541
violation: 0.008065122293033364
violation: 0.007491622249907343
violation: 0.006878074480788151
violation: 0.00627775233458084
violation: 0.005832422550823737
violation: 0.0053615156354635774
violation: 0.004979107737046629
violation: 0.0046754127809737435
violation: 0.004386239599175303
violation: 0.004099228320491612
violation: 0.0038452699357458733
violation: 0.0036638411251082663
violation: 0.0034941095574229208
violation: 0.0033169179092000633
violation: 0.0031541312804225265
violation: 0.0029707057733692754
violation: 0.002794473

### silhouette

In [270]:
from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

print(__doc__)

# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1)  # For reproducibility

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

Automatically created module for IPython interactive environment
For n_clusters = 2 The average silhouette_score is : 0.7049787496083262
For n_clusters = 3 The average silhouette_score is : 0.5882004012129721
For n_clusters = 4 The average silhouette_score is : 0.6505186632729437
For n_clusters = 5 The average silhouette_score is : 0.56376469026194
For n_clusters = 6 The average silhouette_score is : 0.4504666294372765
